# Get CIFAR-10 Data

In directory ~/data/wcl-data: 
- wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
- tar -xvf cifar-10-python.tar.gz

Follow unzipping instructions: [https://www.cs.toronto.edu/~kriz/cifar.html]

In [16]:
import os

os.getcwd()
os.chdir('/home/sshad/data/wcl-data/cifar-10-batches-py')
os.getcwd() 

'/home/sshad/data/wcl-data/cifar-10-batches-py'

In [24]:
os.listdir()

['data_batch_4',
 'readme.html',
 'test_batch',
 'data_batch_3',
 'data_batch_2',
 'data_batch_1',
 'data_batch_5',
 'batches.meta']

In [23]:
batches =[file for file in os.listdir() if '_batch' in file]
batches

['data_batch_4',
 'test_batch',
 'data_batch_3',
 'data_batch_2',
 'data_batch_1',
 'data_batch_5']

In [26]:
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [29]:
batch1 = unpickle('data_batch_1')